In [1]:
import os

from semantic_kernel.agents import AgentGroupChat, ChatCompletionAgent
from semantic_kernel.agents.strategies.selection.kernel_function_selection_strategy import KernelFunctionSelectionStrategy
from semantic_kernel.agents.strategies.termination.kernel_function_termination_strategy import KernelFunctionTerminationStrategy
#from semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion import AzureChatCompletion

from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.agents.open_ai.azure_assistant_agent import AzureAssistantAgent
from semantic_kernel.contents.chat_history import ChatHistory
from semantic_kernel.contents.utils.author_role import AuthorRole
from semantic_kernel.functions.kernel_function_decorator import kernel_function
from semantic_kernel.functions.kernel_function_from_prompt import KernelFunctionFromPrompt
from semantic_kernel.kernel import Kernel

In [2]:
# define agents names

QUERY_ANALYZER_AGENT = "analyzer"
TEXT_AGENT = "text_generator"
IMAGES_AGENT = "image_generator"

In [12]:
def create_kernel_with_chat_completion(service_id: str) -> Kernel:
    kernel = Kernel()
    kernel.add_service(AzureChatCompletion(
        deployment_name=os.getenv('GPT_DEPLOYMENT_NAME'),
        api_key=os.getenv('AZURE_OPENAI_KEY'),
        api_version=os.getenv('AZURE_OPENAI_VERSION'),
        service_id=service_id
        
    ))

    return kernel


In [4]:
chat_completion_client = AzureChatCompletion(
    api_key=os.getenv('AZURE_OPENAI_KEY'),
    deployment_name=os.getenv('GPT_DEPLOYMENT_NAME'),
    base_url=os.getenv('AZURE_OPENAI_ENDPOINT'),
    api_version=os.getenv('AZURE_OPENAI_VERSION')
)

In [5]:
os.getenv('AZURE_OPENAI_VERSION')

'2024-05-01-preview'

In [6]:
analyzer_instructions = '''Your job is to read carefully the user query and understand what are the user needs.
Determine what kind of document will be helpful in order to help the user learns of the requested topic.

Your options can be:
- text generation and pdf document
- text and image generation and pdf document
- text and image generation and ppt presentation
- only image generation
- flow chart diagram
- concept map
etc.,

Based on query determine what option will fit the best to help the user to learn.
'''

query_analyzer_agent = ChatCompletionAgent(
    service = chat_completion_client,
    kernel = create_kernel_with_chat_completion(),
    name = QUERY_ANALYZER_AGENT,
    instructions = analyzer_instructions
)

In [7]:
text_generator_agent = ChatCompletionAgent(
    service = chat_completion_client,
    kernel = create_kernel_with_chat_completion(),
    name = TEXT_AGENT,
    instructions = "Return: Hey this is a sample text agent"
)

image_generator_agent = ChatCompletionAgent(
    service = chat_completion_client,
    kernel = create_kernel_with_chat_completion(),
    name = IMAGES_AGENT,
    instructions = "Return: Hey this is a sample images agent"
)

In [8]:
selection_function = KernelFunctionFromPrompt(
    function_name="selection",
    prompt=f"""Determine which participant takes the next turn in a conversation based on the most recent participant.
    State only the name of the participant to take the next turn.
    Consider that the {QUERY_ANALYZER_AGENT} always starts first and from there it can choose
    the {IMAGES_AGENT}, the {TEXT_AGENT} or both.
    """
)

In [9]:
TERMINATION_KEYWORD = "good"

termination_function = KernelFunctionFromPrompt(
    function_name="termination",
    prompt=f"""Examine the final response and check if the user is satisfied with the responses. 
    If so, respond with this single word without explanation: {TERMINATION_KEYWORD}
    
RESPONSE:
{{{{$history}}}}
    """
)

In [10]:
chat = AgentGroupChat(
    agents=[query_analyzer_agent, text_generator_agent, image_generator_agent],
    selection_strategy=KernelFunctionSelectionStrategy(
        function=selection_function,
        kernel=create_kernel_with_chat_completion(),
        result_parser=lambda result: str(result.value[0]) if result.value is not None else QUERY_ANALYZER_AGENT,
        agent_variable_name="agents",
        history_variable_name="history"
    ),

    termination_strategy=KernelFunctionTerminationStrategy(
        agents=[query_analyzer_agent],
        function=termination_function,
        kernel=create_kernel_with_chat_completion(),
        result_parser=lambda result: TERMINATION_KEYWORD in str(result.value[0]).lower(),
        history_variable_name="history",
        maximum_iterations=10
    )
)

In [13]:
is_complete : bool = False

while not is_complete:
    user_input = input("User:> ")
    if not user_input:
        print("Empty input")
        continue
    
    if user_input.lower() == "good":
        print("exiting conversation...")
        is_complete= True
        break

    #await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=user_input))

    async for response in chat.invoke():
        print(f" # {response.role} - {response.name or '*'} - {response.content}")
    
    if chat.is_complete:
        print("chat is complete")
        is_complete = True
        break



ServiceResponseException: ("<class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service failed to complete the prompt", NotFoundError("Error code: 404 - {'error': {'code': '404', 'message': 'Resource not found'}}"))